In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 24: It Hangs in the Balance ---</h2><p>It's Christmas Eve, and Santa is loading up the sleigh for this year's deliveries.  However, there's one small problem: he can't get the sleigh to balance.  If it isn't balanced, he can't defy physics, and nobody gets presents this year.</p>
<p>No pressure.</p>
<p>Santa has provided you a list of the weights of every package he needs to fit on the sleigh.  The packages need to be split into <em>three groups of exactly the same weight</em>, and every package has to fit.  The first group goes in the passenger compartment of the sleigh, and the second and third go in containers on either side.  Only when all three groups weigh exactly the same amount will the sleigh be able to fly.  Defying physics has rules, you know!</p>
<p>Of course, that's not the only problem.  The first group - the one going in the passenger compartment - needs <em>as few packages as possible</em> so that Santa has some legroom left over.  It doesn't matter how many packages are in either of the other two groups, so long as all of the groups weigh the same.</p>
<p>Furthermore, Santa tells you, if there are multiple ways to arrange the packages such that the fewest possible are in the first group, you need to choose the way where the first group has <em>the smallest quantum entanglement</em> to reduce the chance of any <span title="Santa does not elaborate on what he means by this, but the cringe he makes indicates that it wouldn't be pretty.">"complications"</span>.  The quantum entanglement of a group of packages is the <a href="https://en.wikipedia.org/wiki/Product_%28mathematics%29">product</a> of their weights, that is, the value you get when you multiply their weights together.  Only consider quantum entanglement if the first group has the fewest possible number of packages in it and all groups weigh the same amount.</p>
<p>For example, suppose you have ten packages with weights <code>1</code> through <code>5</code> and <code>7</code> through <code>11</code>.  For this situation, some of the unique first groups, their quantum entanglements, and a way to divide the remaining packages are as follows:</p>
<pre><code>Group 1;             Group 2; Group 3
11 9       (QE= 99); 10 8 2;  7 5 4 3 1
10 9 1     (QE= 90); 11 7 2;  8 5 4 3
10 8 2     (QE=160); 11 9;    7 5 4 3 1
10 7 3     (QE=210); 11 9;    8 5 4 2 1
10 5 4 1   (QE=200); 11 9;    8 7 3 2
10 5 3 2   (QE=300); 11 9;    8 7 4 1
10 4 3 2 1 (QE=240); 11 9;    8 7 5
9 8 3      (QE=216); 11 7 2;  10 5 4 1
9 7 4      (QE=252); 11 8 1;  10 5 3 2
9 5 4 2    (QE=360); 11 8 1;  10 7 3
8 7 5      (QE=280); 11 9;    10 4 3 2 1
8 5 4 3    (QE=480); 11 9;    10 7 2 1
7 5 4 3 1  (QE=420); 11 9;    10 8 2
</code></pre>
<p>Of these, although <code>10 9 1</code> has the smallest quantum entanglement (<code>90</code>), the configuration with only two packages, <code>11 9</code>, in the passenger compartment gives Santa the most legroom and wins.  In this situation, the quantum entanglement for the ideal configuration is therefore <code>99</code>.  Had there been two configurations with only two packages in the first group, the one with the smaller quantum entanglement would be chosen.</p>
<p>What is the <em>quantum entanglement</em> of the first group of packages in the ideal configuration?</p>
</article>


In [2]:
from copy import deepcopy
from heapq import heappop, heappush
from math import prod
from typing import Generator, Iterable, Iterator
from more_itertools import set_partitions, take


weights = list(range(1, 5 + 1)) + list(range(7, 11 + 1))


def pack_bf(weights: list[int]) -> list[list[int]]:
    return min(
        (
            sorted((a, b, c), key=len)
            for a, b, c in set_partitions(weights, 3)
            if sum(a) == sum(b) == sum(c)
        ),
        key=lambda t: (len(t[0]), prod(t[0])),
    )


def pack(weights: list[int]):
    heap = [(0, 1, 0, tuple(sorted(weights, reverse=True)), (tuple(), tuple()))]
    target = sum(weights) // 3

    while heap:
        length, qe, gr, ws, groups = heappop(heap)
        # print(length, qe, gr, groups[0], ws, groups[1])
        som = sum(groups[gr])
        if som == target:
            if gr == 1:
                return groups + (ws,)
            heappush(heap, (length, qe, 1, ws, groups))
        elif som < target:
            for i, w in enumerate(ws):
                if not groups[gr] or w <= groups[gr][-1]:
                    if gr == 0:
                        groups1 = (groups[0] + (w,), groups[1])
                    else:
                        groups1 = (groups[0], groups[1] + (w,))
                    qe1 = qe if gr else qe * w
                    l1 = length if gr else length + 1
                    heappush(heap, (l1, qe1, gr, ws[:i] + ws[i + 1 :], groups1))


# print("pack_bf", pack_bf(weights))

print(pack(weights))

((11, 9), (10, 8, 2), (7, 5, 4, 3, 1))


In [3]:
weights_f = [
    1,
    3,
    5,
    11,
    13,
    17,
    19,
    23,
    29,
    31,
    37,
    41,
    43,
    47,
    53,
    59,
    67,
    71,
    73,
    79,
    83,
    89,
    97,
    101,
    103,
    107,
    109,
    113,
]

res = pack(weights_f)
print("Part I:", prod(res[0]))

Part I: 10439961859


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>10439961859</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>That's weird... the sleigh still isn't balancing.</p>
<p>"Ho ho ho", Santa muses to himself. "I forgot the trunk".</p>
<p>Balance the sleigh again, but this time, separate the packages into <em>four groups</em> instead of three.  The other constraints still apply.</p>
<p>Given the example packages above, this would be some of the new unique first groups, their quantum entanglements, and one way to divide the remaining packages:</p>
<pre><code>
11 4    (QE=44); 10 5;   9 3 2 1; 8 7
10 5    (QE=50); 11 4;   9 3 2 1; 8 7
9 5 1   (QE=45); 11 4;   10 3 2;  8 7
9 4 2   (QE=72); 11 3 1; 10 5;    8 7
9 3 2 1 (QE=54); 11 4;   10 5;    8 7
8 7     (QE=56); 11 4;   10 5;    9 3 2 1
</code></pre>
<p>Of these, there are three arrangements that put the minimum (two) number of packages in the first group: <code>11 4</code>, <code>10 5</code>, and <code>8 7</code>.  Of these, <code>11 4</code> has the lowest quantum entanglement, and so it is selected.</p>
<p>Now, what is the <em>quantum entanglement</em> of the first group of packages in the ideal configuration?</p>
</article>

</main>


In [4]:
def pack_II(weights: list[int]):
    heap = [
        (0, 1, 0, tuple(sorted(weights, reverse=True)), (tuple(), tuple(), tuple()))
    ]
    target = sum(weights) // 4

    while heap:
        length, qe, gr, ws, groups = heappop(heap)
        # print(length, qe, gr, groups, ws)
        som = sum(groups[gr])
        if som == target:
            if gr == 2:
                return groups + (ws,)
            heappush(heap, (length, qe, gr + 1, ws, groups))
        elif som < target:
            for i, w in enumerate(ws):
                if not groups[gr] or w <= groups[gr][-1]:
                    if gr == 0:
                        groups1 = (groups[0] + (w,), groups[1], groups[2])
                    elif gr == 1:
                        groups1 = (groups[0], groups[1] + (w,), groups[2])
                    else:
                        groups1 = (groups[0], groups[1], groups[2] + (w,))
                    qe1 = qe if gr else qe * w
                    l1 = length if gr else length + 1
                    heappush(heap, (l1, qe1, gr, ws[:i] + ws[i + 1 :], groups1))


# print("pack_bf", pack_bf(weights))

print(pack_II(weights))

((11, 4), (10, 5), (9, 3, 2, 1), (8, 7))


In [5]:
res = pack_II(weights_f)
print("Part I:", prod(res[0]))

Part I: 72050269


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>72050269</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="24/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22It+Hangs+in+the+Balance%22+%2D+Day+24+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F24&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22It+Hangs+in+the+Balance%22+%2D+Day+24+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F24';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


Argument of type "list[tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]]" cannot be assigned to parameter "\_\_heap" of type "list[_T@heappush]" in function "heappush"
"list[tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]]" is incompatible with "list[tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]] | tuple[Literal[0], Literal[1], Literal[1], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]]"
Type parameter "\_T@list" is invariant, but "tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]" is not the same as "tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]] | tuple[Literal[0], Literal[1], Literal[1], tuple[int, ...],

list[tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]]
list[tuple[Literal[0], Literal[1], Literal[0], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]] | tuple[Literal[0], Literal[1], Literal[1], tuple[int, ...], tuple[tuple[int, ...], tuple[int, ...]]]]
